<a href="https://colab.research.google.com/github/Srinath460/Datascience/blob/main/Bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install pandas
!pip3 install numpy
!pip3 install kaggle
!pip3 install opendatasets

In [6]:
import opendatasets as od

od.download(
    "https://www.kaggle.com/datasets/mfaisalqureshi/spam-email")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: srinathjuturu
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mfaisalqureshi/spam-email


100%|██████████| 207k/207k [00:00<00:00, 35.7MB/s]

In [7]:
ls spam-email


spam.csv


In [8]:
import pandas as pd

inputDF=pd.read_csv('spam-email/spam.csv')
inputDF.count()


Category    5572
Message     5572
dtype: int64

In [ ]:
inputDF.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
inputDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [14]:
#Convert Message column to lower case
import string
inputDF.Message=inputDF.Message.str.lower()
inputDF.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [15]:
inputDF.Category=inputDF.Category.apply(lambda x: True if x=='spam' else False)
inputDF.head()

,Category,Message
0,False,"go until jurong point, crazy.. available only ..."
1,False,ok lar... joking wif u oni...
2,True,free entry in 2 a wkly comp to win fa cup fina...
3,False,u dun say so early hor... u c already then say...
4,False,"nah i don't think he goes to usf, he lives aro..."


In [16]:
#splitting into training and testing datasets
train_df=inputDF.iloc[:int(len(inputDF)*0.7)]

test_df=inputDF.iloc[int(len(inputDF)*0.7):]

In [17]:
train_df.head()

,Category,Message
0,False,"go until jurong point, crazy.. available only ..."
1,False,ok lar... joking wif u oni...
2,True,free entry in 2 a wkly comp to win fa cup fina...
3,False,u dun say so early hor... u c already then say...
4,False,"nah i don't think he goes to usf, he lives aro..."


In [18]:
test_df.head()

,Category,Message
3900,False,ü mean it's confirmed... i tot they juz say on...
3901,False,okie
3902,False,that depends. how would you like to be treated...
3903,False,"right on brah, see you later"
3904,False,waiting in e car 4 my mum lor. u leh? reach ho...


In [19]:
fract_spam_texts=train_df.Category.mean()
print(fract_spam_texts)

0.13307692307692306


In [20]:
train_spam_words=' '.join(train_df[train_df.Category == True].Message).split(' ')

train_non_spam_words=' '.join(train_df[train_df.Category == False].Message).split(' ')

In [21]:
common_words=set(train_spam_words).intersection(set(train_non_spam_words))

print(len(train_spam_words))
print(len(train_non_spam_words))

12441
48680


In [22]:
train_spam_words_bow=dict()
for word in common_words:
  train_spam_words_bow[word]=train_spam_words.count(word)/len(train_spam_words)

In [23]:
train_non_spam_words_bow=dict()
for word in common_words:
  train_non_spam_words_bow[word]=train_non_spam_words.count(word)/len(train_non_spam_words)

**Bayes Theorum**

P(A|B) = P(B|A).P(A)/P(B)

=> P(A|B) ∝ P(B|A).P(A) -> 1

Taking logs on both sides so we don't loose data due decimal precisions

log(P(A|B)) ∝ log(P(B|A)) + log(P(A)) -> 2

**Conditional Independance**

P(A1,A2,A3....,An|B) = πP(Ai|B) -> 3

Taking logs on both sides so we don't loose data due decimal precisions

log(P(A1,A2,A3....,An|B)) = Σlog(P(Ai|B) -> 4

From 2 and 4

log(P(B|A1,A2,A3....,An)) ∝ Σlog(P(Ai|B) + log(B)


**Example**

log(P(spam|urgent call this number)) ∝ log(P(urgent|spam)) + log(P(call|spam) + log(P(this|spam)) + log(P(number|spam)) + log(P(spam))


In [24]:
#Function to predict if the message is spam or not

import numpy as np
def predict_spam_mail(sentance_list):
  valid_words=[ word for word in sentance_list if word in common_words ]

  # for w in valid_words:
  #   print(f'word={w} prob of spam={train_spam_words_bow[w]} prob of non_spam={train_non_spam_words_bow[w]} ')

  probability_of_spam=[ train_spam_words_bow[word] for word in valid_words ]

  probability_of_non_spam=[ train_non_spam_words_bow[word] for word in valid_words ]

  spam_score=sum([np.log(prob) for prob in probability_of_spam]) + np.log(fract_spam_texts)

  non_spam_score=sum([np.log(prob) for prob in probability_of_non_spam]) + np.log(1-fract_spam_texts)

  # print(f'spam_score={spam_score}')
  # print(f'non_spam_score={non_spam_score}')

  return(spam_score>=non_spam_score)



In [25]:
predict_spam_mail('urgent call this number as soon as possible'.split())

True

In [26]:
predict_spam_mail('do you want to go to a movie tonight'.split())

False

In [27]:
predict_spam_mail('Lets go out tonight'.split())

False

In [30]:
def correct_prediction_percentage(df):
  count=0
  for row in df.iterrows():
    predicted_output=predict_spam_mail(row[1]['Message'].split())
    if(predicted_output==row[1]['Category']):
      count+=1
  print(count/len(df)*100)


In [32]:
#Training dataset prediction

correct_prediction_percentage(train_df)

97.05128205128204


In [33]:
#Testing dataset prediction

correct_prediction_percentage(test_df)

96.29186602870813
